# Addgene API Gemini Demo

This notebook demonstrates how to use the `gemini_client` helpers together with the Addgene scraping code to design plasmids from natural language descriptions.

In [ ]:
!pip install -q --use-deprecated=legacy-resolver -r requirements.txt

Set your Google Gemini API key:

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY_HERE"

### 1. Import helpers

In [ ]:
from gemini_client import extract_search_params, design_plasmid
from main import scrape_plasmids, ADDGENE_PLASMID_CATALOG_PATH
import httpx, asyncio

### 2. Describe the plasmid you need

In [ ]:
description = "E. coli expression vector with ampicillin resistance"
protein_seq = "MKKFFDSRREQMMARAA"

### 3. Generate search parameters using Gemini

In [ ]:
params = extract_search_params(description)
params

### 4. Search Addgene for matching plasmids

In [ ]:
async def search(params):
    async with httpx.AsyncClient(base_url="https://www.addgene.org", http2=True) as client:
        resp = await client.get(ADDGENE_PLASMID_CATALOG_PATH, params=params)
        resp.raise_for_status()
        return scrape_plasmids(resp.text)

plasmids = asyncio.run(search(params))
plasmids[:3]

### 5. Design a plasmid sequence using Gemini

In [ ]:
design = design_plasmid(description, protein_seq)
print(design)